# Global Illumination Replica

Contains script to apply global illumination change to the replica dataset.

In [ ]:
# Settings
from PIL import Image, ImageEnhance
import os
import numpy as np
from tqdm import tqdm
import pathlib
import shutil

# Set the directory containing the images you want to modify
directory = "../Datasets/Replica/"
output_dir = "../Datasets/Replica_tmp/"

# Set the desired brightness reduction value (0-255, where 0 is completely dark and 255 is completely bright)
brightness_reduction = 150
period:int = 100
freq = np.pi * 2 / period

In [ ]:
# find all maps
maps = [x for x in os.listdir(directory) if not x.startswith(".") and os.path.isdir(os.path.join(directory, x))]

for map in maps:
    print(f"Processing map {map}...")

    map_path = os.path.join(directory, map, "results")
    output_map_path = os.path.join(output_dir, map, "results")
    # copy traj.txt
    pathlib.Path(output_map_path).mkdir(parents=True, exist_ok=True)
    shutil.copy(os.path.join(map_path, "../traj.txt"), os.path.join(output_map_path, "../traj.txt"))

    # Loop through all the files in the directory
    img_files = [x for x in os.listdir(map_path)]
    for img_file in tqdm(img_files):
        image_path = os.path.join(map_path, img_file)
        image_save_path = os.path.join(output_map_path, img_file)
        # just copy depth maps
        if img_file.startswith("depth"):
            shutil.copy(image_path, image_save_path)            
            continue
        
        # Open the image file
        img = Image.open(image_path)
    
        # Reduce the brightness
        enhancer = ImageEnhance.Brightness(img)
        id = int(img_file.split(".")[0][5:])
        # sin with cutoff
        img = enhancer.enhance(1 + min(0, brightness_reduction*np.sin(freq*id)/255))
    
        # Save the modified image with a new filename
        img.save(image_save_path, "JPEG")
        

print("Done!")